# Order Analysis project

In this project, I utilize Python to analyze orders data.
>1. Load data from Kaggle API
>2. Load data into Jupyter Notebook
>3. Use Pandas to clean and analyze data
>4. Create a connection to MSSQL
>5. Create a connection from MSSQL to Jupyter Notebook


© Harison Nagisvaran

In [108]:
!pip install kaggle
import pandas as pd

In [19]:
!kaggle datasets download akshatagrawal9431/shipment-orders-dataset -f orders_data.csv


Dataset URL: https://www.kaggle.com/datasets/akshatagrawal9431/shipment-orders-dataset
License(s): CC0-1.0
orders_data.csv.zip: Skipping, found more recently modified local copy (use --force to force download)


In [20]:
#Extract file from zip file

import zipfile
file = zipfile.ZipFile('orders_data.csv.zip')
file.extractall() 

file.close() #close file

In [21]:
df = pd.read_csv('orders_data.csv', na_values = ['Not Available', 'unknown'])
df.head()

,Order Id,Order Date,Ship Mode,Segment,Country,City,State,Postal Code,Region,Category,Sub Category,Product Id,cost price,List Price,Quantity,Discount Percent
0,1,01-03-2023,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Bookcases,FUR-BO-10001798,240,260,2,2
1,2,15-08-2023,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Chairs,FUR-CH-10000454,600,730,3,3
2,3,10-01-2023,Second Class,Corporate,United States,Los Angeles,California,90036,West,Office Supplies,Labels,OFF-LA-10000240,10,10,2,5
3,4,18-06-2022,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Furniture,Tables,FUR-TA-10000577,780,960,5,2
4,5,13-07-2022,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Office Supplies,Storage,OFF-ST-10000760,20,20,2,5


In [22]:
df.shape #file structure check

(9994, 16)

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Order Id          9994 non-null   int64 
 1   Order Date        9994 non-null   object
 2   Ship Mode         9988 non-null   object
 3   Segment           9994 non-null   object
 4   Country           9994 non-null   object
 5   City              9994 non-null   object
 6   State             9994 non-null   object
 7   Postal Code       9994 non-null   int64 
 8   Region            9994 non-null   object
 9   Category          9994 non-null   object
 10  Sub Category      9994 non-null   object
 11  Product Id        9994 non-null   object
 12  cost price        9994 non-null   int64 
 13  List Price        9994 non-null   int64 
 14  Quantity          9994 non-null   int64 
 15  Discount Percent  9994 non-null   int64 
dtypes: int64(6), object(10)
memory usage: 1.2+ MB


In [24]:
df["Ship Mode"].unique()

array(['Second Class', 'Standard Class', nan, 'First Class', 'Same Day'],
      dtype=object)

In [25]:
df['Selling Price'] = df['List Price'] - df['List Price']*(df['Discount Percent']/100)
df['Selling Price']

0       254.8
1       708.1
2         9.5
3       940.8
4        19.0
        ...  
9989     28.8
9990     86.4
9991    254.8
9992     29.1
9993    232.8
Name: Selling Price, Length: 9994, dtype: float64

In [26]:
df.head()

,Order Id,Order Date,Ship Mode,Segment,Country,City,State,Postal Code,Region,Category,Sub Category,Product Id,cost price,List Price,Quantity,Discount Percent,Selling Price
0,1,01-03-2023,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Bookcases,FUR-BO-10001798,240,260,2,2,254.8
1,2,15-08-2023,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Chairs,FUR-CH-10000454,600,730,3,3,708.1
2,3,10-01-2023,Second Class,Corporate,United States,Los Angeles,California,90036,West,Office Supplies,Labels,OFF-LA-10000240,10,10,2,5,9.5
3,4,18-06-2022,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Furniture,Tables,FUR-TA-10000577,780,960,5,2,940.8
4,5,13-07-2022,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Office Supplies,Storage,OFF-ST-10000760,20,20,2,5,19.0


In [27]:
#Calculating the profit

df['Profit'] = df['Selling Price'] - df['cost price']

In [28]:
df.rename(columns = {'Profit':'Unit_Profit', 'Selling Price':"Unit_Selling_Price"}, inplace = True)

In [29]:
#Change data type of Order date column from Object to datetime
df['Order Date'] = pd.to_datetime(df['Order Date'])
df.dtypes

C:\Users\hnaud\AppData\Local\Temp\ipykernel_20984\774907106.py:2: UserWarning: Parsing '15-08-2023' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  df['Order Date'] = pd.to_datetime(df['Order Date'])
C:\Users\hnaud\AppData\Local\Temp\ipykernel_20984\774907106.py:2: UserWarning: Parsing '18-06-2022' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  df['Order Date'] = pd.to_datetime(df['Order Date'])
C:\Users\hnaud\AppData\Local\Temp\ipykernel_20984\774907106.py:2: UserWarning: Parsing '13-07-2022' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  df['Order Date'] = pd.to_datetime(df['Order Date'])
C:\Users\hnaud\AppData\Local\Temp\ipykernel_20984\774907106.py:2: UserWarning: Parsing '13-03-2022' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  df['Order Date'] = pd.to_datetime(df['Or

Order Id                       int64
Order Date            datetime64[ns]
Ship Mode                     object
Segment                       object
Country                       object
City                          object
State                         object
Postal Code                    int64
Region                        object
Category                      object
Sub Category                  object
Product Id                    object
cost price                     int64
List Price                     int64
Quantity                       int64
Discount Percent               int64
Unit_Selling_Price           float64
Unit_Profit                  float64
dtype: object

In [30]:
#Drop unncecessary columns
df.drop(columns = ['cost price', 'List Price','Discount Percent'], inplace = True)

In [31]:
df.head()

,Order Id,Order Date,Ship Mode,Segment,Country,City,State,Postal Code,Region,Category,Sub Category,Product Id,Quantity,Unit_Selling_Price,Unit_Profit
0,1,2023-01-03,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Bookcases,FUR-BO-10001798,2,254.8,14.8
1,2,2023-08-15,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Chairs,FUR-CH-10000454,3,708.1,108.1
2,3,2023-10-01,Second Class,Corporate,United States,Los Angeles,California,90036,West,Office Supplies,Labels,OFF-LA-10000240,2,9.5,-0.5
3,4,2022-06-18,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Furniture,Tables,FUR-TA-10000577,5,940.8,160.8
4,5,2022-07-13,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Office Supplies,Storage,OFF-ST-10000760,2,19.0,-1.0


In [32]:
#Calclating total profit per order
df['Total profit'] = df['Quantity']*df['Unit_Profit']

### Exporting df to SQL using SQLAlCHEMY

In [33]:
import sqlalchemy as sal

engine = sal.create_engine('mssql://Manda/Projects?driver=ODBC+Driver+17+for+SQL+Server')

In [34]:
conn = engine.connect()
conn

In [35]:
df.to_sql('orders_data', con=conn, index = False, if_exists = 'replace') 

-1

### Question 1: Find top 10 Highest profit generating products

In [36]:
df['Product Id'].value_counts()

OFF-PA-10001970    19
TEC-AC-10003832    18
FUR-FU-10004270    16
FUR-CH-10001146    15
TEC-AC-10003628    15
                   ..
TEC-PH-10003535     1
TEC-MA-10002937     1
OFF-AR-10004974     1
TEC-AC-10004814     1
OFF-ST-10001627     1
Name: Product Id, Length: 1862, dtype: int64

In [37]:
df.groupby(['Product Id']).sum(['Total Profit'])

,Order Id,Postal Code,Quantity,Unit_Selling_Price,Unit_Profit,Total profit
Product Id,,,,,,
FUR-BO-10000112,3513,60653,9,805.1,45.1,405.9
FUR-BO-10000330,13658,256733,10,1036.4,146.4,560.0
FUR-BO-10000362,18886,184360,14,2089.4,189.4,638.6
FUR-BO-10000468,35749,357215,21,706.1,86.1,380.6
FUR-BO-10000711,9310,120635,12,830.9,40.9,345.9
...,...,...,...,...,...,...
TEC-PH-10004912,21739,166396,11,493.8,53.8,203.8
TEC-PH-10004922,31470,296048,19,1037.3,57.3,259.2
TEC-PH-10004924,13937,105326,8,48.2,-1.8,-6.0


In [38]:
df.groupby(['Product Id']).sum(['Total Profit']).sort_values(by = ['Total profit'], ascending = False).head(10)


,Order Id,Postal Code,Quantity,Unit_Selling_Price,Unit_Profit,Total profit
Product Id,,,,,,
TEC-CO-10004722,28222,194875,20,59514.0,5644.0,24816.0
TEC-MA-10002412,2698,32216,6,21734.4,3624.4,21746.4
OFF-BI-10000545,57937,302599,48,18249.0,1959.0,17867.7
TEC-CO-10001449,41444,428939,38,18151.2,2631.2,15948.0
FUR-CH-10002024,44560,448062,39,21096.2,2246.2,13930.7
OFF-BI-10003527,61859,562064,31,26525.3,3435.3,12792.9
TEC-PH-10001459,38070,346769,39,13380.7,1290.7,11481.9
TEC-MA-10000822,4419,283475,18,16192.5,1612.5,10102.3
FUR-TA-10000198,28061,249689,27,9559.7,1229.7,10015.1


### Create a column for Total Sales value for each order

In [39]:
df.head()

,Order Id,Order Date,Ship Mode,Segment,Country,City,State,Postal Code,Region,Category,Sub Category,Product Id,Quantity,Unit_Selling_Price,Unit_Profit,Total profit
0,1,2023-01-03,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Bookcases,FUR-BO-10001798,2,254.8,14.8,29.6
1,2,2023-08-15,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Chairs,FUR-CH-10000454,3,708.1,108.1,324.3
2,3,2023-10-01,Second Class,Corporate,United States,Los Angeles,California,90036,West,Office Supplies,Labels,OFF-LA-10000240,2,9.5,-0.5,-1.0
3,4,2022-06-18,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Furniture,Tables,FUR-TA-10000577,5,940.8,160.8,804.0
4,5,2022-07-13,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Office Supplies,Storage,OFF-ST-10000760,2,19.0,-1.0,-2.0


In [40]:
df['Total Sales'] = df['Unit_Selling_Price'] * df['Quantity']
df['Total Sales']

0        509.6
1       2124.3
2         19.0
3       4704.0
4         38.0
         ...  
9989      86.4
9990     172.8
9991     509.6
9992     116.4
9993     465.6
Name: Total Sales, Length: 9994, dtype: float64

### Question 3: List all distinct cities where orders have been shipped

In [41]:
df['City'].unique()

array(['Henderson', 'Los Angeles', 'Fort Lauderdale', 'Concord',
       'Seattle', 'Fort Worth', 'Madison', 'West Jordan', 'San Francisco',
       'Fremont', 'Philadelphia', 'Orem', 'Houston', 'Richardson',
       'Naperville', 'Melbourne', 'Eagan', 'Westland', 'Dover',
       'New Albany', 'New York City', 'Troy', 'Chicago', 'Gilbert',
       'Springfield', 'Jackson', 'Memphis', 'Decatur', 'Durham',
       'Columbia', 'Rochester', 'Minneapolis', 'Portland', 'Saint Paul',
       'Aurora', 'Charlotte', 'Orland Park', 'Urbandale', 'Columbus',
       'Bristol', 'Wilmington', 'Bloomington', 'Phoenix', 'Roseville',
       'Independence', 'Pasadena', 'Newark', 'Franklin', 'Scottsdale',
       'San Jose', 'Edmond', 'Carlsbad', 'San Antonio', 'Monroe',
       'Fairfield', 'Grand Prairie', 'Redlands', 'Hamilton', 'Westfield',
       'Akron', 'Denver', 'Dallas', 'Whittier', 'Saginaw', 'Medina',
       'Dublin', 'Detroit', 'Tampa', 'Santa Clara', 'Lakeville',
       'San Diego', 'Brentwood', 'Cha

### Question 4: Calculate the total selling price and profits for all orders

In [42]:
df['Order value'] = df['Unit_Selling_Price']*df['Quantity']
df.head()

,Order Id,Order Date,Ship Mode,Segment,Country,City,State,Postal Code,Region,Category,Sub Category,Product Id,Quantity,Unit_Selling_Price,Unit_Profit,Total profit,Total Sales,Order value
0,1,2023-01-03,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Bookcases,FUR-BO-10001798,2,254.8,14.8,29.6,509.6,509.6
1,2,2023-08-15,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Chairs,FUR-CH-10000454,3,708.1,108.1,324.3,2124.3,2124.3
2,3,2023-10-01,Second Class,Corporate,United States,Los Angeles,California,90036,West,Office Supplies,Labels,OFF-LA-10000240,2,9.5,-0.5,-1.0,19.0,19.0
3,4,2022-06-18,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Furniture,Tables,FUR-TA-10000577,5,940.8,160.8,804.0,4704.0,4704.0
4,5,2022-07-13,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Office Supplies,Storage,OFF-ST-10000760,2,19.0,-1.0,-2.0,38.0,38.0


### Question 5: Find all orders from the 'Technology' category that were shipped using 'Second Class' ship mode, ordered by order date

In [43]:
df[(df['Category']=='Technology')& (df['Ship Mode']=='Second Class')]

,Order Id,Order Date,Ship Mode,Segment,Country,City,State,Postal Code,Region,Category,Sub Category,Product Id,Quantity,Unit_Selling_Price,Unit_Profit,Total profit,Total Sales,Order value
19,20,2022-11-01,Second Class,Consumer,United States,San Francisco,California,94109,West,Technology,Phones,TEC-PH-10001949,3,203.7,33.7,101.1,611.1,611.1
26,27,2022-08-02,Second Class,Consumer,United States,Los Angeles,California,90049,West,Technology,Accessories,TEC-AC-10003027,3,87.3,7.3,21.9,261.9,261.9
161,162,2023-11-27,Second Class,Consumer,United States,Philadelphia,Pennsylvania,19134,East,Technology,Accessories,TEC-AC-10003657,2,47.5,7.5,15.0,95.0,95.0
181,182,2022-06-25,Second Class,Corporate,United States,Decatur,Illinois,62521,Central,Technology,Accessories,TEC-AC-10004659,7,397.7,17.7,123.9,2783.9,2783.9
182,183,2022-10-01,Second Class,Home Office,United States,Monroe,Louisiana,71203,South,Technology,Phones,TEC-PH-10003273,4,475.0,5.0,20.0,1900.0,1900.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9891,9892,2023-05-21,Second Class,Corporate,United States,Milwaukee,Wisconsin,53209,Central,Technology,Accessories,TEC-AC-10003116,7,116.4,6.4,44.8,814.8,814.8
9892,9893,2023-02-01,Second Class,Corporate,United States,Milwaukee,Wisconsin,53209,Central,Technology,Phones,TEC-PH-10002262,5,291.0,31.0,155.0,1455.0,1455.0
9949,9950,2022-07-01,Second Class,Corporate,United States,Indianapolis,Indiana,46203,Central,Technology,Accessories,TEC-AC-10004568,3,77.6,7.6,22.8,232.8,232.8
9950,9951,2022-03-19,Second Class,Corporate,United States,Indianapolis,Indiana,46203,Central,Technology,Accessories,TEC-AC-10001714,1,38.8,-1.2,-1.2,38.8,38.8


### Question 6: Find the average order value

In [44]:
import numpy as np

np.mean(df['Order value'])

1108.5979787872702

### Question 7: Find the city with the highest total quantity of products ordered

In [45]:
df.groupby(['City']).sum(['Quantity']).sort_values(by = ['Quantity'], ascending = False).head(1)

,Order Id,Postal Code,Quantity,Unit_Selling_Price,Unit_Profit,Total profit,Total Sales,Order value
City,,,,,,,,
New York City,4450928,9168909,3417,247205.7,20925.7,103852.6,1218342.6,1218342.6


### Question 8: Rank orders in each region by quantity in descending order

In [46]:
df.groupby(['Region']).sum(['Quantity']).sort_values(by=['Quantity'], ascending = False).head()

,Order Id,Postal Code,Quantity,Unit_Selling_Price,Unit_Profit,Total profit,Total Sales,Order value
Region,,,,,,,,
West,15971838,293739752,12266,699858.6,62558.6,305289.6,3467409.6,3467409.6
East,14073919,50171698,10618,654320.0,58220.0,295773.8,3257983.8,3257983.8
Central,11685963,151786150,8780,483668.6,46378.6,236121.2,2387881.2,2387881.2
South,8213295,55875052,6209,378011.5,38011.5,202743.6,1966053.6,1966053.6


### Question 9: List all orders placed in the first quarter of any year, including the total cost for these costs

In [47]:
df['Month']= df['Order Date'].dt.month
df[df['Month'].isin([1,2,3])]

,Order Id,Order Date,Ship Mode,Segment,Country,City,State,Postal Code,Region,Category,Sub Category,Product Id,Quantity,Unit_Selling_Price,Unit_Profit,Total profit,Total Sales,Order value,Month
0,1,2023-01-03,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Bookcases,FUR-BO-10001798,2,254.8,14.8,29.6,509.6,509.6,1
5,6,2022-03-13,NaN,Consumer,United States,Los Angeles,California,90032,West,Furniture,Furnishings,FUR-FU-10001487,7,48.5,-1.5,-10.5,339.5,339.5,3
7,8,2022-01-25,Standard Class,Consumer,United States,Los Angeles,California,90032,West,Technology,Phones,TEC-PH-10002275,6,864.5,4.5,27.0,5187.0,5187.0,1
8,9,2023-03-23,NaN,Consumer,United States,Los Angeles,California,90032,West,Office Supplies,Binders,OFF-BI-10003910,3,19.6,-0.4,-1.2,58.8,58.8,3
10,11,2023-03-31,NaN,Consumer,United States,Los Angeles,California,90032,West,Furniture,Tables,FUR-TA-10001539,9,1658.7,188.7,1698.3,14928.3,14928.3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9967,9968,2022-02-10,Standard Class,Consumer,United States,Plainfield,New Jersey,7060,East,Office Supplies,Binders,OFF-BI-10004209,5,39.2,9.2,46.0,196.0,196.0,2
9977,9978,2023-03-12,Standard Class,Home Office,United States,Los Angeles,California,90032,West,Office Supplies,Fasteners,OFF-FA-10003467,7,9.6,-0.4,-2.8,67.2,67.2,3
9987,9988,2023-01-04,Standard Class,Corporate,United States,Athens,Georgia,30605,South,Technology,Accessories,TEC-AC-10001539,1,77.6,-2.4,-2.4,77.6,77.6,1
9989,9990,2023-02-18,Second Class,Consumer,United States,Miami,Florida,33180,South,Furniture,Furnishings,FUR-FU-10001889,3,28.8,-1.2,-3.6,86.4,86.4,2


### Question 10: Find top 10 highest profit generating products

In [54]:
df.groupby(['Product Id']).sum(['Total profit']).sort_values(by=['Total profit'], ascending = False).head(10)

,Order Id,Postal Code,Quantity,Unit_Selling_Price,Unit_Profit,Total profit,Total Sales,Order value,Month
Product Id,,,,,,,,,
TEC-CO-10004722,28222,194875,20,59514.0,5644.0,24816.0,245056.0,245056.0,47
TEC-MA-10002412,2698,32216,6,21734.4,3624.4,21746.4,130406.4,130406.4,10
OFF-BI-10000545,57937,302599,48,18249.0,1959.0,17867.7,163777.7,163777.7,45
TEC-CO-10001449,41444,428939,38,18151.2,2631.2,15948.0,107388.0,107388.0,65
FUR-CH-10002024,44560,448062,39,21096.2,2246.2,13930.7,120090.7,120090.7,51
OFF-BI-10003527,61859,562064,31,26525.3,3435.3,12792.9,97082.9,97082.9,80
TEC-PH-10001459,38070,346769,39,13380.7,1290.7,11481.9,113041.9,113041.9,38
TEC-MA-10000822,4419,283475,18,16192.5,1612.5,10102.3,89622.3,89622.3,27
FUR-TA-10000198,28061,249689,27,9559.7,1229.7,10015.1,77845.1,77845.1,37


### Question 11: Find top 3 highest selling products in each region according to quantity

In [59]:
df.groupby(['Region','Product Id']).sum(['Quantity']).reset_index().sort_values(by=['Region','Quantity'], ascending = [True,False]).head(3)

,Region,Product Id,Order Id,Postal Code,Quantity,Unit_Selling_Price,Unit_Profit,Total profit,Total Sales,Order value,Month
461,Central,OFF-BI-10000301,34082,452734,34,106.3,-3.7,-19.5,570.5,570.5,45
474,Central,OFF-BI-10000756,32806,486684,33,58.3,8.3,76.5,446.5,446.5,53
470,Central,OFF-BI-10000546,22342,383143,29,38.6,-1.4,-8.2,221.8,221.8,47


### Question 12: Total Sale in each region

In [69]:
df.groupby(['Region']).sum('Total Sales')['Total Sales']

Region
Central    2387881.2
East       3257983.8
South      1966053.6
West       3467409.6
Name: Total Sales, dtype: float64

### Question 14: Find Top 5 highest selling products in each region according to Sales


In [83]:
df.groupby(['Region','Product Id']).sum(['Total Sales']).reset_index().sort_values(by =['Total Sales'], ascending = False)

,Region,Product Id,Order Id,Postal Code,Quantity,Unit_Selling_Price,Unit_Profit,Total profit,Total Sales,Order value,Month
3691,South,TEC-MA-10002412,2698,32216,6,21734.4,3624.4,21746.4,130406.4,130406.4,10
469,Central,OFF-BI-10000545,17681,123422,18,10132.7,1072.7,13247.5,125827.5,125827.5,5
2556,East,TEC-CO-10004722,13241,48855,11,29099.0,2249.0,8261.0,106421.0,106421.0,29
1166,Central,TEC-CO-10004722,6827,47905,5,16975.0,2975.0,14875.0,84875.0,84875.0,8
2565,East,TEC-MA-10001047,10619,18736,11,13767.0,1607.0,9989.0,81549.0,81549.0,5
...,...,...,...,...,...,...,...,...,...,...,...
1669,East,OFF-AR-10000369,1232,14215,2,0.0,0.0,0.0,0.0,0.0,5
678,Central,OFF-FA-10003059,4201,139355,4,0.0,0.0,0.0,0.0,0.0,13
145,Central,FUR-FU-10001918,8184,60653,1,0.0,0.0,0.0,0.0,0.0,6
594,Central,OFF-BI-10004141,8457,138860,5,0.0,0.0,0.0,0.0,0.0,13


### Question 15: Find month over month growth comparison for 2022 and 2023 sales 

In [84]:
df.head(2)

,Order Id,Order Date,Ship Mode,Segment,Country,City,State,Postal Code,Region,Category,Sub Category,Product Id,Quantity,Unit_Selling_Price,Unit_Profit,Total profit,Total Sales,Order value,Month
0,1,2023-01-03,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Bookcases,FUR-BO-10001798,2,254.8,14.8,29.6,509.6,509.6,1
1,2,2023-08-15,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Chairs,FUR-CH-10000454,3,708.1,108.1,324.3,2124.3,2124.3,8


In [91]:
df['year'] = df['Order Date'].dt.year

In [93]:
a = df.groupby(['Month','year']).sum('Total Sales').reset_index()
a.head(2)

,Month,year,Order Id,Postal Code,Quantity,Unit_Selling_Price,Unit_Profit,Total profit,Total Sales,Order value
0,1,2022,2174234,24669713,1632,102196.1,9686.1,47480.9,483080.9,483080.9
1,1,2023,2191058,23201106,1616,86104.8,7294.8,33387.3,405977.3,405977.3


In [95]:
pivot_table = a.pivot_table(index = 'Month', columns = 'year', values = 'Total Sales', aggfunc = 'sum')
pivot_table

year,2022,2023
Month,,
1,483080.9,405977.3
2,411145.4,647618.3
3,370927.1,477912.2
4,409544.8,431460.7
5,500027.8,443481.2
6,457905.9,299893.9
7,400229.4,508056.9
8,430209.9,460226.4
9,449362.6,458050.6


In [99]:
pivot_table['YoY Growth'] = pivot_table[2022] - pivot_table[2023]

### Reading data from SQL Server in Python

In [106]:
# Windows authentication

import pyodbc as p

# Properly formatted connection string
cnxn = p.connect(
    r'Driver ={SQL SERVER};'
    r'Server=manda;'
    r'Trusted_Connection=yes;'
    r'Database=Projects;'
)


InterfaceError: ('IM002', '[IM002] [Microsoft][ODBC Driver Manager] Data source name not found and no default driver specified (0) (SQLDriverConnect)')